# Tutorial of rgeoda v0.0.3


by: Xun Li 10/10/2019

rgeoda is a R package that wraps all core functions of spatial data analysis in GeoDa and libgeoda. Unlike the desktop software GeoDa, libgeoda is a non-UI and feature focused C++ library that is designed for programmers to do spatial data analysis using their favoriate programming languages (R, Python, Java etc.). It also aims to be easily integratd with other libraries, softwares or systems on different platforms.

## 1. Install rgeoda

Like GeoDa desktop software, `rgeoda` are avaiable to different platforms including: Mac, Linux and Windows. 

### Mac OSX and Linux

In R console, use install.packages() function to install rgeoda from its source pacakge at: https://github.com/lixun910/rgeoda/archive/0.0.3.tar.gz

```r
install.packages("https://github.com/lixun910/rgeoda/archive/0.0.3.tar.gz")
# or the development version
# devtools::install_github("lixun910/rgeoda")
```

### Windows

In R console, use install.packages() function to install rgeoda from its source pacakge at: https://github.com/lixun910/rgeoda/releases/download/0.0.3/rgeoda_0.0.3.zip

```r
install.packages("https://github.com/lixun910/rgeoda/releases/download/0.0.3/rgeoda_0.0.3.zip")
# or the development version
# devtools::install_github("lixun910/rgeoda")
```
Install rgeoda on windows from source package is not recommended. You would try if you know how to deal with [R devtools](https://www.r-project.org/nosvn/pandoc/devtools.html) on windows.

### Load rgeoda library in R

If everything installed without error, you should be able to load rgeoda:


In [1]:
library(rgeoda)

## 2. Load Spatial Data


The data formats that rgeoda can read directly includes: ESRI Shapefile, MapInfo File, CSV, GML, GPX, KML, GeoJSON, TopoJSON, OpenFileGDB, GFT Google Fusion Tables, CouchDB 

Note: in this tutorial, we only tested loading ESRI shapefiles using rgeoda v0.0.3. Please create a ticket in rgeoda's [repository](https://github.com/lixun910/rgeoda/issues) if you experience any issues when loading spatial data. 

For example, to load the ESRI Shapefile `Guerry.shp` comes with the package:


In [3]:
guerry_path <- system.file("extdata", "Guerry.shp", package = "rgeoda")


In [4]:
guerry <- geoda_open(guerry_path)

The `geoda_open` function returns a geoda object, which can be used to access the meta-data, fields, and columns of the input dataset.

#### 2.1 Attributes of `geoda` object

* n_cols
* n_obs
* field_names
* field_types
* table (data.frame)
* GetUndefinesCol(col_name)

In [5]:
cat("\nnumber of columns:", guerry$n_cols)
cat("\nnumber of observations:", guerry$n_obs)
cat("\nfield names:", guerry$field_names)
cat("\nfield types:", guerry$field_types)


number of columns: 26
number of observations: 85
field names: CODE_DE COUNT AVE_ID_ dept Region Dprtmnt Crm_prs Crm_prp Litercy Donatns Infants Suicids MainCty Wealth Commerc Clergy Crm_prn Infntcd Dntn_cl Lottery Desertn Instrct Prsttts Distanc Area Pop1831
field types: string numeric numeric integer string string integer integer integer integer integer integer integer integer integer integer integer integer integer integer integer integer integer numeric integer numeric

### 2.2 Access Table Data


The `geoda` instance has a data.frame attribute `table`, which stores the data loaded from the dataset. To get the values of "Crm_prp" column:


In [6]:
crm_prp <- guerry$table$Crm_prp
cat("\nfirs 10 values of Crm_prp:", crm_prp[1:10])


firs 10 values of Crm_prp: 15890 5521 7925 7289 8174 10263 8847 9597 4086 10431

## 3. Spatial Weights

Spatial weights are central components in spatial data analysis. The spatial weights represents the possible spatial interaction between observations in space. Like GeoDa desktop software, `rgeoda` provides a rich variety of methods to create several different types of spatial weights:

* Contiguity Based Weights: `queen_weights()`, `rook_weights()`
* Distance Based Weights: `distance_weights()`
* K-Nearest Neighbor Weights: `knn_weights()`
* Kernel Weights: `distance_weights()` and `knn_weights()` with kernel parameters

### 3.1 Queen Contiguity Weights

To create a Queen contiguity weights, we can call pygeoda's function 
```r
gda_queen(gda, order=1, include_lower_order = False, precision_threshold = 0)
``` 
by passing the GeoDa object `guerry` we just created: 


In [7]:
queen_w <- queen_weights(guerry)
queen_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9cf49b020>

Field "is_symmetric":
[1] TRUE
Field "sparsity":
[1] 0
Field "density":
[1] 5.813149
Field "min_neighbors":
[1] 2
Field "max_neighbors":
[1] 8
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 4.941176
Field "median_neighbors":
[1] 5

The function `queen_weights()` returns an instance of 
`Weight` object. One can access the meta data of the spatial
weights by accessing the attributes of `GeoDaWeight` object:

#### Attributes of `Weight` object

* weight_type
* is_symmetric
* sparsity
* density
* min_nbrs
* max_nbrs
* mean_nbrs
* median_nbrs
* bool HasIsolates()
* [] GetNeighbors(idx)
* double SpatialLag(idx, [data])
* SaveToFile()

We can also access the details of the weights: e.g. list the neighbors of a specified observation, which is very helpful in exploratory spatial data analysis (which is focused in next tutorial):


In [8]:
nbrs <- queen_w$GetNeighbors(0)
cat("\nNeighbors of 0-st observation are:", nbrs)


Neighbors of 0-st observation are: 35 36 66 68

We can also compute the spatial lag of a specified observation by passing the values of the selected variable:

In [9]:
lag0 <- queen_w$SpatialLag(0, crm_prp)
cat("\nSpatial lag of 0-st observation is:", lag0)


Spatial lag of 0-st observation is: 7899.25

### 3.2 Rook Contiguity Weights

To create a Rook contiguity weights, we can call pygeoda's function 
```r
rook_weights(gda, order=1,include_lower_order=False, precision_threshold = 0)
``` 
by passing the GeoDa object `guerry` we just created: 


In [10]:
rook_w <- rook_weights(guerry)
rook_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9d19c9280>

Field "is_symmetric":
[1] TRUE
Field "sparsity":
[1] 0
Field "density":
[1] 5.813149
Field "min_neighbors":
[1] 2
Field "max_neighbors":
[1] 8
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 4.941176
Field "median_neighbors":
[1] 5

The weights we created are in memory, which makes it straight forward for spatial data analysis and also are good for programming your application. To save the weights to a file, we need to call GeoDaWeight's function 
```r
SaveToFile(ofname, layer_name, id_name, id_vec)
```

The `layer_name` is the layer name of loaded dataset. For a ESRI shapefile, the layer name is the file name without the suffix (e.g. Guerry). 

The `id_name` is a key (column name), which means the associated column contains unique values, that makes sure that the weights are connected to the correct observations in the data table. 

The `id_vec` is the actual column data of `id_name`, it could be a tuple of integer or string values.

For example, in Guerry dataset, the column "CODE_DE" can be used as a key to save a weights file:


In [11]:
rook_w$SaveToFile('./Guerry_r.gal', 'Guerry', 'CODE_DE', guerry$GetIntegerCol('CODE_DE'))

[1] TRUE

Then, we should find the file "Guerry_r.gal" in the output directory.

### 3.3 Distance Based Weights

To create a Distance based weights, we can call rgeoda's function 
```r
gda_distance_weights(gda, dist_thres, power=1.0,  is_inverse=False, is_arc=False, is_mile=True)
``` 
by passing the GeoDa object `guerry` we just created and the value of distance threshold. Like GeoDa, rgeoda provides a function to help you find a optimized distance threshold that guarantees that every observation has at least one neighbor:

```r
min_distthreshold(GeoDa gda, bool is_arc = False, bool is_mile = True)
```

In [12]:
dist_thres <- min_distthreshold(guerry)
dist_w <- distance_weights(guerry, dist_thres)
dist_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9d8738d20>

Field "is_symmetric":
[1] FALSE
Field "sparsity":
[1] 0
Field "density":
[1] 4.346021
Field "min_neighbors":
[1] 1
Field "max_neighbors":
[1] 7
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 3.694118
Field "median_neighbors":
[1] 4

### 3.4 K-Nearest Neighbor Weights

A special case of distance based weights is K-Nearest neighbor weights, in which every obersvation will have exactly k neighbors. To create a KNN weights, we can call pygeoda's function:
```r
gda_knn_weights(gda, k, power = 1.0,is_inverse = False, is_arc = False, is_mile = True)
```

For example, to create a 6-nearest neighbor weights using Guerry dataset:

In [13]:
knn6_w <- knn_weights(guerry, 6)
knn6_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9d19454a0>

Field "is_symmetric":
[1] FALSE
Field "sparsity":
[1] 0
Field "density":
[1] 7.058824
Field "min_neighbors":
[1] 6
Field "max_neighbors":
[1] 6
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 6
Field "median_neighbors":
[1] 6

### 3.5 Kernel Weights

Kernel weights apply kernel function to determine the distance decay in the derived continuous weights kernel. The kernel weights are defined as a function K(z) of the ratio between the distance dij from i to j, and the bandwidth hi, with z=dij/hi. 

The kernl functions include

* triangular
* uniform 
* quadratic
* epanechnikov
* quartic
* gaussian

Two functions are provided in rgeoda to create kernel weights.

**Kernel Weights with adaptive bandwidth**

To create a kernel weights with fixed bandwith:

In [14]:
bandwidth <- min_distthreshold(guerry)
kernel_w <- kernel_weights(guerry, bandwidth, "uniform")
kernel_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9d90e2270>

Field "is_symmetric":
[1] FALSE
Field "sparsity":
[1] 0
Field "density":
[1] 4.346021
Field "min_neighbors":
[1] 1
Field "max_neighbors":
[1] 7
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 3.694118
Field "median_neighbors":
[1] 4

Besides the options `is_inverse`, `power`, `is_arc` and `is_mile` that are the same with the distance based weights, this kernel weights function has another option:
```
use_kernel_diagonals	
(optional) FALSE (default) or TRUE, apply kernel on the diagonal of weights matrix
```

** Kernel Weights with adaptive bandwidth**

To create a kernel weights with adaptive bandwidth or using max Knn distance as bandwidth:

In [15]:
adptkernel_w = kernel_knn_weights(guerry, 6, "uniform")

adptkernel_w

Reference class object of class "Weight"
Field "gda_w":
An object of class "_p_GeoDaWeight"
Slot "ref":
<pointer: 0x7fa9d90e89e0>

Field "is_symmetric":
[1] FALSE
Field "sparsity":
[1] 0
Field "density":
[1] 7.058824
Field "min_neighbors":
[1] 6
Field "max_neighbors":
[1] 6
Field "num_obs":
[1] 85
Field "mean_neighbors":
[1] 6
Field "median_neighbors":
[1] 6

This kernel weights function two more options:
```
adaptive_bandwidth	
(optional) TRUE (default) or FALSE: TRUE use adaptive bandwidth calculated using distance of k-nearest neithbors, FALSE use max distance of all observation to their k-nearest neighbors

use_kernel_diagonals	
(optional) FALSE (default) or TRUE, apply kernel on the diagonal of weights matrix
```


## 4 Spatial Data Analysis

### 4.1 Local Spatial Autocorrelation


rgeoda 0.0.3 provids following methods for univariate local spatial autocorrelation statistics:


* Local Moran: local_moran()
* Local Geary: local_geary()
* Local Getis-Ord statistics: local_g() and local_gstar()
* Local Join Count: local_joincount()


Methods for bivariate and multivariate local spatial autocorrelation statistics, as well as global spatial autocorrelation satatistics, will be included in next release of rgeoda.

In this tutorial, we will only introduce how to call these methods using pygeoda. For more information about the local spatial autocorrelation statisticis, please read: http://geodacenter.github.io/workbook/6a_local_auto/lab6a.html. 

#### 4.1.1 Local Moran

The Local Moran statistic is a method to identify local clusters and local spatial outliers. For example, we can call  function `local_moran()` with the created Queen weights and the data “crm_prp” as input parameters:


In [16]:
lisa <- local_moran(queen_w, crm_prp)


The `local_moran()` function will return a `lisa` object, which we can call its public functions to access the results of lisa computation.

For example, we can call the function `GetLISAValues()` to get the values of local Moran:


In [17]:
lms <- lisa$GetLISAValues()
lms

[1]  0.0154319783  0.3270633224  0.0212952962  0.0046105448 -0.0028342407
 [6]  0.4149377158 -0.1379463091  0.0998657692  0.2823176310  0.1218745112
[11] -0.0951205417  0.0326111938  0.3878324535  1.1888723840 -0.6452792226
[16] -0.3096492740  0.3662775143  2.0375343539 -0.0050154800  0.0697110572
[21]  2.2720355722  0.2886391551 -0.0070189726 -0.0009906605  0.9517469793
[26]  0.7648322095  0.0432039929 -0.0045362842 -0.0964911860  0.0952360887
[31]  0.0100895206 -0.0109401003 -0.0544245927 -0.0345553975  0.0887531916
[36]  0.0954232056  0.0383617454 -0.1776714441  0.1638208393  1.2309154898
[41]  2.9077203402 -0.0396340261  0.4458735227  0.2491123240  0.0905643622
[46] -0.6212977834 -0.0308773407  0.0375870399  0.2270376112 -0.0918254739
[51] -0.0112400086  0.1085584763 -0.0055113129 -0.0027097589  0.7235016208
[56]  0.0163129939  0.4246564560  0.3787307767 -0.0597158189  0.5050011802
[61]  2.7632125275  0.0656510809  0.1771899330 -0.0572150317  0.4943795537
[66]  0.2870386197 -1.4593300774 -0.0055305930  0.4895990016 -0.0324125662
[71]  2.1366499813  0.9041683235  0.7053678641  1.4098290925  0.0051652159
[76]  0.2238144189 -0.1621373954  0.0195632289 -0.3233724187 -0.0337778226
[81]  0.0118189869 -0.1164679533 -0.5699624657 -0.0859634996  0.2085373916

To get the pseudo-p values of significance of local Moran computation:


In [18]:
pvals <- lisa$GetPValues()
pvals

[1] 0.414 0.123 0.001 0.474 0.452 0.087 0.243 0.326 0.299 0.303 0.237 0.460
[13] 0.258 0.018 0.199 0.188 0.131 0.004 0.456 0.342 0.041 0.143 0.429 0.412
[25] 0.002 0.026 0.480 0.400 0.121 0.350 0.481 0.441 0.338 0.120 0.267 0.050
[37] 0.072 0.221 0.282 0.022 0.029 0.494 0.075 0.054 0.200 0.025 0.359 0.346
[49] 0.280 0.375 0.487 0.267 0.441 0.428 0.135 0.314 0.050 0.002 0.085 0.297
[61] 0.001 0.233 0.243 0.134 0.116 0.342 0.015 0.334 0.069 0.303 0.040 0.132
[73] 0.001 0.002 0.470 0.007 0.213 0.387 0.123 0.403 0.455 0.352 0.011 0.322
[85] 0.135

To get the cluster indicators of local Moran computation:


In [19]:
cats <- lisa$GetClusterIndicators()
cats

[1] 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 2 2 0 0 0 0 0 0 0 0 0 1 0 0
[39] 0 1 1 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 2 2 0 0 1 0 0 0 0 0 3 0 0 0 2 0 2 2 0 2
[77] 0 0 0 0 0 0 3 0 0

The predefined values of the indicators of LISA cluster are:
```
0 Not significant
1 High-High
2 Low-Low
3 High-Low
4 Low-High
5 Neighborless
6 Undefined
```
which can be accessed via function `GetLabels()`:

In [20]:
lbls <- lisa$GetLabels()
lbls

[1] "Not significant" "High-High"       "Low-Low"         "High-Low"       
[5] "Low-High"        "Undefined"       "Isolated"

By default, the `local_moran()` function will run with some default parameters, e.g.:
```
permutation number: 999
seed for random number generator: 123456789
```
, which are identical to GeoDa desktop software so that we can replicate the results as using  GeoDa software. It is also easy to change the paremter and re-run the LISA computation by calling Run() function. 

For example, re-run the above local Moran example using 9999 permutations

In [21]:
lisa$SetPermutations(9999)
lisa$Run()

NULL

NULL

Then, we can use the same `lisa` object to get the new results after 9999 permutations:


In [22]:
pvals <- lisa$GetPValues()
pvals

[1] 0.4187 0.1265 0.0004 0.4679 0.4545 0.0728 0.2312 0.3071 0.3115 0.3088
[11] 0.2187 0.4803 0.2623 0.0113 0.2000 0.1797 0.1267 0.0026 0.4565 0.3517
[21] 0.0450 0.1367 0.4344 0.4231 0.0024 0.0366 0.4741 0.4164 0.1293 0.3310
[31] 0.4893 0.4371 0.3350 0.1376 0.2723 0.0567 0.0753 0.2221 0.2559 0.0192
[41] 0.0216 0.4918 0.0739 0.0564 0.2193 0.0199 0.3667 0.3339 0.2764 0.3783
[51] 0.4679 0.2683 0.4495 0.4312 0.1423 0.3158 0.0612 0.0009 0.0900 0.2861
[61] 0.0001 0.2378 0.2518 0.1295 0.1262 0.3482 0.0105 0.3152 0.0692 0.2984
[71] 0.0260 0.1331 0.0001 0.0001 0.4683 0.0067 0.2197 0.3718 0.1392 0.4054
[81] 0.4420 0.3462 0.0039 0.3136 0.1288

rgeoda uses GeoDa’s C++ code, in which multi-threading is used to accelerate the computation of LISA. We can specify how many threads to run the computation:


In [23]:
lisa$SetThreads(4)
lisa$Run()
lisa$GetPValues()

NULL

NULL

[1] 0.4187 0.1265 0.0004 0.4679 0.4545 0.0728 0.2312 0.3071 0.3115 0.3088
[11] 0.2187 0.4834 0.2686 0.0102 0.2024 0.1795 0.1218 0.0025 0.4590 0.3588
[21] 0.0476 0.1456 0.4344 0.4231 0.0024 0.0366 0.4741 0.4164 0.1293 0.3310
[31] 0.4893 0.4371 0.3350 0.1424 0.2751 0.0564 0.0713 0.2202 0.2672 0.0186
[41] 0.0214 0.4960 0.0721 0.0589 0.2288 0.0189 0.3759 0.3217 0.2812 0.3735
[51] 0.4695 0.2743 0.4518 0.4286 0.1471 0.3222 0.0647 0.0016 0.0955 0.2840
[61] 0.0001 0.2388 0.2465 0.1242 0.1275 0.3493 0.0120 0.3158 0.0719 0.2979
[71] 0.0291 0.1386 0.0001 0.0003 0.4627 0.0058 0.2168 0.3761 0.1386 0.4119
[81] 0.4631 0.3421 0.0045 0.3138 0.1257

#### 4.1.2 Local Geary

Local Geary is a type of LISA that focuses on squared differences/dissimilarity. A small value of the local geary statistics suggest positive spatial autocorrelation, whereas large values suggest negative spatial autocorrelation. 

For example, we can call the function `local_geary()` with the created Queen weights and the data “crm_prp” as input parameters:


In [24]:
geary_crmprs <- local_geary(queen_w, crm_prp)


To get the cluster indicators of the local Geary computation:


In [25]:
geary_crmprs$GetPValues()


[1] 0.398 0.027 0.025 0.126 0.017 0.314 0.610 0.141 0.284 0.110 0.559 0.462
[13] 0.211 0.236 0.249 0.229 0.069 0.041 0.205 0.020 0.029 0.178 0.369 0.112
[25] 0.024 0.027 0.092 0.420 0.488 0.051 0.377 0.253 0.217 0.049 0.110 0.151
[37] 0.147 0.584 0.327 0.107 0.009 0.129 0.135 0.190 0.031 0.086 0.084 0.292
[49] 0.299 0.418 0.053 0.431 0.178 0.077 0.092 0.294 0.116 0.321 0.573 0.204
[61] 0.196 0.264 0.084 0.616 0.131 0.408 0.058 0.088 0.282 0.264 0.040 0.085
[73] 0.059 0.002 0.711 0.543 0.620 0.057 0.335 0.411 0.197 0.625 0.010 0.364
[85] 0.265

#### 4.1.3 Local Getis-Ord Statistics

There are two types of local Getis-Ord statistics: one is computing a ratio of the weighted average of the values in the neighboring locations, not including the value at the location; while another type of statistic includes the value at the location in both numerator and denominator.

A value larger than the mean suggests a high-high cluster or hot spot, a value smaller than the mean indicates a low-low cluster or cold spot.

For example, we can call the function `local_g()` with the created Queen weights and the data “crm_prp” as input parameters:


In [27]:
localg_crmprs <- local_g(queen_w, crm_prp)

To get the cluster indicators of the local G computation:


In [28]:
localg_crmprs$GetClusterIndicators()


[1] 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 2 2 0 0 0 0 0 0 0 0 0 1 0 0
[39] 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 2 0 0 1 0 0 0 0 0 1 0 0 0 2 0 2 2 0 2
[77] 0 0 0 0 0 0 1 0 0

To get the pseudo-p values of the local G computation:


In [29]:
localg_crmprs$GetPValues()


[1] 0.414 0.123 0.001 0.474 0.452 0.087 0.243 0.326 0.299 0.303 0.237 0.460
[13] 0.258 0.018 0.199 0.188 0.131 0.004 0.456 0.342 0.041 0.143 0.429 0.412
[25] 0.002 0.026 0.479 0.400 0.121 0.350 0.481 0.441 0.338 0.120 0.266 0.050
[37] 0.072 0.221 0.282 0.022 0.029 0.494 0.075 0.054 0.200 0.025 0.359 0.346
[49] 0.280 0.375 0.487 0.266 0.441 0.428 0.135 0.314 0.050 0.002 0.085 0.297
[61] 0.001 0.233 0.243 0.134 0.116 0.340 0.015 0.334 0.069 0.303 0.039 0.132
[73] 0.001 0.002 0.470 0.007 0.213 0.387 0.123 0.403 0.455 0.352 0.011 0.322
[85] 0.135

For the second type of local Getis-Ord statistics, we can call the function `local_gstar()` with the created Queen weights and the data “crm_prp” as input parameters:


In [30]:
localgstar_crmprs <- local_gstar(queen_w, crm_prp)


#### 4.1.4 Local Join Count


Local Join Count is a method to identify local clusters for binary data by using a local version of the so-called BB join count statistic. The statistic is only meaningful for those observations with value 1. 

For example, we can load the columbus dataset, and call the function `local_joincount()` with a Queen weights and the data “nsa”, which is a set of binary (0,1) values, as input parameters:


In [31]:
columbus_path <- system.file("extdata", "columbus.shp", package = "rgeoda")
columbus <- geoda_open(columbus_path)

columbus_w <- queen_weights(columbus)
localjc_crmprs <- local_joincount(columbus_w, columbus$table$nsa)

To get the cluster indicators of the local Join Count computation:


In [32]:
localjc_crmprs$GetClusterIndicators()


[1] 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[39] 0 0 0 0 0 0 0 0 0 0 0

To get the pseudo-p values of the local Join Count  computation:


In [33]:
localjc_crmprs$GetPValues()


[1] 0.213 0.070 0.017 0.010 0.001 0.234 0.018 0.007 0.006 0.275 0.017 0.008
[13] 0.043 0.004 0.002 0.001 0.147 0.049 0.087 0.001 0.001 0.372 0.001 0.450
[25] 0.287 0.249 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
[37] 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
[49] 0.001

To get the number of neighbors of the local Join Count computation:


In [34]:
localjc_crmprs$GetNumNeighbors()


[1]  2  3  4  5  8  2  5  6  8  4  5  6  4  6  6  8  3  4  3 10  3  6  3  7  8
[26]  6  4  9  7  5  3  4  4  4  7  5  6  6  3  5  3  2  6  5  4  2  2  4  3

### 4.2 Spatial Clustering

 
Spatial clustering aims to group of a large number of geographic areas or points into a smaller number of regions based on similiarities in one or more variables. Spatially constrained clustering is needed when clusters are required to be spatially contiguous. 

In GeoDa, there are three different approaches explicitly incorporate the contiguity constraint in the optimization process: SKATER, Redcap and Max-p. More more details, please check: http://geodacenter.github.io/workbook/8_spatial_clusters/lab8.html All of these methods are included in rgeoda 0.0.3.

For example, to apply spatial clustering on the Guerry dataset, we use the queen weights to define the spatial contiguity and select 6 variables for similarity measure: "Crm_prs", "Crm_prp", "Litercy", "Donatns", "Infants", "Suicids". 

The following code is used to get a 2D data vector for the selected variables:


In [35]:
data <- list(guerry$table$Crm_prs, guerry$table$Crm_prp, guerry$table$Litercy, guerry$table$Donatns, guerry$table$Infants, guerry$table$Suicids)

#### 4.2.1 SKATER

The Spatial C(K)luster Analysis by Tree Edge Removal(SKATER) algorithm introduced by Assuncao et al. (2006) is based on the optimal pruning of a minimum spanning tree that reflects the contiguity structure among the observations. It provides an optimized algorithm to prune to tree into several clusters that their values of selected variables are as similar as possible.

The rgeoda's SKATER function is: 
```r
skater(k, w, data, distance_method='euclidean', bound_vals = [],  min_bound = 0, random_seed=123456789)
```

For example, to create 4 spatially contiguous clusters using Guerry dataset, the queen weights and the values of the 6 selected variables:

In [37]:
guerry_clusters <- skater(4, queen_w, data)
guerry_clusters

[[1]]
 [1] 15 74 16 55 60 39 68 33 17 82 81  0  2 40 20 80

[[2]]
 [1] 46 50 34 38 69 47 58 19 32 41 53 26

[[3]]
 [1] 23 79  3 29 61 21 44 11 28 13 30 35 76 77 43  9 27 45 31 78  4 10 66 37  5
[26] 14  7 63 62

[[4]]
 [1] 49 52 72 84  8 57 56 59 42  1 25 51 48 54 64 75 18 83 73 36 24 71  6 67 65
[26] 70 22 12

This `skater()` function returns a 2D list, which represents 4 clusters. Each cluster is composed by several contiguity areas, e.g. 15, 74, 16, 55, 60, 39, 68, 33, 17, 82, 81, 0, 2, 40, 20, 80

rgeoda also provides utility functions to compute some descriptive statistics of the clustering results, e.g. to compute the ratio of between to total sum of squares:



In [38]:
betweenss <- between_sumofsquare(guerry_clusters, data)
totalss <- total_sumofsquare( data)
ratio <-  betweenss / totalss
cat("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.3156447

#### 4.2.2 REDCAP

REDCAP (Regionalization with dynamically constrained agglomerative clustering and partitioning) is developed by D. Guo (2008). Like SKATER, REDCAP starts from building a spanning tree with 3 different ways (single-linkage, average-linkage, and the complete-linkage). The single-linkage way leads to build a minimum spanning tree. Then,REDCAP provides 2 different ways (first‐order and full-order constraining) to prune the tree to find clusters. The first-order approach with a minimum spanning tree is exactly the same with SKATER. In GeoDa and rgeoda, the following methods are provided:

* First-order and Single-linkage
* Full-order and Complete-linkage
* Full-order and Average-linkage
* Full-order and Single-linkage

For example, to find 4 clusters using the same dataset and weights as above using REDCAP with Full-order and Complete-linkage method:


In [39]:
redcap_clusters <- redcap(4, queen_w, data, "fullorder-completelinkage")
redcap_clusters

betweenss <- between_sumofsquare(redcap_clusters, data)
totalss <- total_sumofsquare( data)
ratio <- betweenss / totalss
cat("The ratio of between to total sum of square:", ratio)

[[1]]
 [1] 15 74 16 55 60 39 68 33 17 82 81  0  2 40 20 80

[[2]]
 [1] 46 50 34 38 69 47 58 19 32 41 53 26

[[3]]
 [1] 23 79  3 29 61 21 44 11 28 13 30 35 76 77 43  9 27 45 31 78  4 10 66 37  5
[26] 14  7 63 62

[[4]]
 [1] 49 52 72 84  8 57 56 59 42  1 25 51 48 54 64 75 18 83 73 36 24 71  6 67 65
[26] 70 22 12

The ratio of between to total sum of square: 0.3156447

#### 4.2.3 Max-p

The so-called max-p regions model (outlined in Duque, Anselin, and Rey 2012) uses a different approach and considers the regionalization problem as an application of integer programming. In addition, the number of regions is determined endogenously.

The algorithm itself consists of a search process that starts with an initial feasible solution and iteratively improves upon it while maintaining contiguity among the elements of each cluster. Like Geoda, rgeoda provides three different heuristic algorithms to find an optimal solution for max-p:

* greedy
* Tabu Search
* Simulated Annealing

Unlike SKATER and REDCAP that one can specify the number of clusters as an input paramter, max-p doesn't allow to specify the number of clusters explicitly, but a constrained variable and the minimum bounding value that each cluster should reach that are used to find an optimized number of clusters.

For example, to use `greedy` algorithm in maxp function with the same dataset and weights as above to find optimal clusters using max-p:

First, we need to specify, for example, every cluster must have population >= 3236.67 thousands people:


In [40]:
bound_vals <- guerry$table$Pop1831
min_bound <- 3236.67 # 10% of Pop1831

Then, we can call the max-p function with "greedy" algorith, the bound values and minimum bound value:


In [41]:
maxp_clusters <- maxp(queen_w, data, bound_vals, min_bound, "greedy")

betweenss <- between_sumofsquare(maxp_clusters, data)
ratio <- betweenss / totalss
cat("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5070181

We can also specify using `tabu search` algorithm in maxp function with the parameter of tabu length:


In [42]:
maxp_tabu_clusters <- maxp(queen_w, data, bound_vals, min_bound, "tabu", tabu_length=95)

betweenss <- between_sumofsquare(maxp_tabu_clusters, data)
ratio <- betweenss / totalss
cat("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5280299

To apply `simulated annealing` algorithm in maxp function with the parameter of cooling rate:


In [43]:
maxp_sa_clusters <- maxp(queen_w, data, bound_vals, min_bound, "sa", cool_rate=0.75)

betweenss <- between_sumofsquare(maxp_sa_clusters, data)
ratio <- betweenss / totalss
cat("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5260249

We can also increase the number of iterations for local search process by specifying the parameter `initial` (default value is 99):

In [44]:
maxp_clusters <- maxp(queen_w, data, bound_vals, min_bound, "greedy", initial=1000)

betweenss <- between_sumofsquare(maxp_clusters, data)
ratio <- betweenss / totalss
cat("Tratio of between to total sum of square:", ratio)

Tratio of between to total sum of square: 0.5260249